# GenealogyBot

Le but de ce bot est d'extraire les données généalogiques provenant l'extraction du recensement de 1832 dans la ville de Lausanne et de créer des pages correspondantes sur wikipast en respectant la bonne syntaxe.

Le document original est accessible sur le [site des archives de Lausanne](https://mipade.mediancer.com/avl/avl_viewer.html?mag=14918). 


## 1 Connexion au bot sur wikipast

Connexion sur wikipast avec l'identifant et le mot de passe créé pour le bot GenealogyBot.

In [16]:
from pywikiapi import Site

user = 'JDezalos@GenealogyBot'
password = 'vu12lg9or06k5pnv1lpur3ou45sd7mat' #critical security breach

site = Site('http://wikipast.epfl.ch/wikipast/api.php') # Définition de l'adresse de l'API
site.no_ssl = True  # Désactivation du https, car pas activé sur wikipast
site.login(user, password)  # Login du bot sur wikipast

## 2 Extraction des données

In [2]:
import pandas as pd

from IPython.display import display
from urllib.parse import quote

#Voir dataset_parser.py

## 3 Vérification d'existance

In [3]:
all_parent = pd.read_csv("../data/filtered.csv")

In [4]:
pages_names = all_parent.apply(lambda x: (x.chef_prenom +" "+ x.chef_nom).title(), axis=1)

results_pages = []
for page in pages_names:
    # Si on veut plus d'info sur les pages qui existent, ajouter à prop
    # Voir http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=query
    for res in site.query_pages(titles=page, prop=["contributors"]):  
        results_pages.append(res)
results_pages

[{'pageid': 1674824,
  'ns': 0,
  'title': 'Louis Collioud',
  'contributors': [{'userid': 30506, 'name': 'JDezalos'}]},
 {'pageid': 349125,
  'ns': 0,
  'title': 'Dominique Dubois',
  'contributors': [{'userid': 30393, 'name': 'Wikidataficator'},
   {'userid': 30394, 'name': 'GallicaSPARQLBot'}]},
 {'pageid': 1674826,
  'ns': 0,
  'title': 'Henry Rochat',
  'contributors': [{'userid': 30506, 'name': 'JDezalos'}]},
 {'pageid': 1674827,
  'ns': 0,
  'title': 'Jean Louis Deprez',
  'contributors': [{'userid': 30506, 'name': 'JDezalos'}]},
 {'pageid': 1674828,
  'ns': 0,
  'title': 'Etienne Baudet',
  'contributors': [{'userid': 30506, 'name': 'JDezalos'}]},
 {'pageid': 1674829,
  'ns': 0,
  'title': 'Jean Daniel Bonnet',
  'contributors': [{'userid': 30506, 'name': 'JDezalos'}]},
 {'pageid': 1674830,
  'ns': 0,
  'title': 'Jacob Rubli',
  'contributors': [{'userid': 30506, 'name': 'JDezalos'}]},
 {'pageid': 1674831,
  'ns': 0,
  'title': 'Jacob Peneveyre',
  'contributors': [{'userid': 3

In [12]:
def on_missing_parent(content):
    prenom = content['chef_prenom'].title()
    nom = content['chef_nom'].title()
    date_naissance = content['chef_annee_naissance']
    titre = '%s %s'%(content['chef_prenom'].title(), content['chef_nom'].title())

    new_page_father(titre, prenom, nom, date_naissance, source)
def on_existing_parent(content):
    prenom = content['chef_prenom'].title()
    nom = content['chef_nom'].title()
    date_naissance = content['chef_annee_naissance']
    titre = '%s %s %s'%(content['chef_prenom'].title(), content['chef_nom'].title(), "- Généalogie")

    new_page_father(titre, prenom, nom, date_naissance, source)

In [ ]:
all_child = pd.read_csv("../data/chef_enfant.csv")

In [ ]:
child_pages_names = all_child.apply(lambda x: (x.enfants_dans_la_commune_prenom +" "+ x.chef_nom).title(), axis=1)

child_results_pages = []
for page in child_pages_names:
    # Si on veut plus d'info sur les pages qui existent, ajouter à prop
    # Voir http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=query
    for res in site.query_pages(titles=page, prop=["contributors"]):  
        child_results_pages.append(res)
child_results_pages

In [ ]:
def check_if_parent_is_missing(content):
    prenom_enfant = content['enfants_dans_la_commune_prenom'].title()
    date_naissance_enfant = content['annee_enfant']
    nom_pere = content['chef_nom'].title()
    prenom_pere = content['chef_prenom'].title()
    nom_mere = content['epouse_nom'].title()
    titre = '%s %s'%(prenom_pere, nom_pere)
    if titre in missing:
        entry_birth_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source)
    else:
        titre = '%s %s %s'%(prenom_pere, nom_pere, "- Généalogie")
        entry_birth_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source)
        
        
def on_missing_child(content):
    prenom_enfant = content['enfants_dans_la_commune_prenom'].title()
    date_naissance_enfant = content['annee_enfant']
    nom_pere = content['chef_nom'].title()
    prenom_pere = content['chef_prenom'].title()
    nom_mere = content['epouse_nom'].title()
    titre = '%s %s'%(content['enfants_dans_la_commune_prenom'].title(), content['chef_nom'].title())

    new_page_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source)
    check_if_parent_is_missing(content)
    
def on_existing_child(content):
    prenom_enfant = content['enfants_dans_la_commune_prenom'].title()
    date_naissance_enfant = content['annee_enfant']
    nom_pere = content['chef_nom'].title()
    prenom_pere = content['chef_prenom'].title()
    nom_mere = content['epouse_nom'].title()
    titre = '%s %s %s'%(content['enfants_dans_la_commune_prenom'].title(), content['chef_nom'], "- Généalogie")
    
    new_page_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source)
    check_if_parent_is_missing(content)

In [18]:
def main():
    missing = [page["title"] for page in results_pages if page.get("missing", False) is True]
    existing_pages = [page for page in results_pages if page.get("missing", False) is False] # existing == not missing donc ça devrait pas être nécessaire
    missing.extend([page["title"] for page in existing_pages if len(page.get("contributors")) == 1 and page.get("contributors")[0].get("name", "") == "JDezalos"])


    all_parent["missing"] = all_parent.apply(lambda x: on_missing_parent(x) if (x.chef_prenom +" "+ x.chef_nom).title() in missing else on_existing_parent(x), axis=1)

    child_missing = [page["title"] for page in child_results_pages if page.get("missing", False) is True]
    child_existing_pages = [page for page in child_results_pages if page.get("missing", False) is False] # existing == not missing donc ça devrait pas être nécessaire
    child_missing.extend([page["title"] for page in child_existing_pages if len(page.get("contributors")) == 1 and page.get("contributors")[0].get("name", "") == "JDezalos"])

    all_child["missing"] = all_child.apply(lambda x: on_missing_child(x) if (x.enfants_dans_la_commune_prenom +" "+ x.chef_nom).title() in child_missing else on_existing_child(x), axis=1)

### 3.1 Création de pages

#### 3.1.1 Création d'une nouvelle page avec l'entrée correspondant à la naissance de la personne

Définition des fonctions "new_page_father" et "new_page_children" permettant de créer respectivement une page pour un chef de famille ou une page pour un enfant avec une entrée pour leur naissance en utilisant la syntaxe correcte :

- pour les chefs de famille : `[[Date]]/ -. [[Naissance]] de [[Prénom Nom]]. [Source]`

- pour les enfants : `[[Date]]/ -. [[Naissance]] de [[Prénom Nom]], enfant de [[Prénom Nom]] et de son épouse née Nom. [Source]`

In [14]:
def new_page_father(titre, prenom, nom, date_naissance, source):
    text = '\n* [[%s]] / -. [[Naissance]] de [[%s %s]]. [%s]\n'%(date_naissance, prenom, nom, source)
    site('delete', titre=titre, token=site.token())
    #site('edit', title=titre, text=text, token=site.token())
    
def new_page_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source):
    if nom_mere == "·":
        text = '\n* [[%d]] / -. [[Naissance]] de [[%s %s]], enfant de [[%s %s]]. [%s]\n'%(date_naissance_enfant, prenom_enfant, nom_pere, prenom_pere, nom_pere, source)
    else:
        text = '\n* [[%d]] / -. [[Naissance]] de [[%s %s]], enfant de [[%s %s]] et de son épouse née %s. [%s]\n'%(date_naissance_enfant, prenom_enfant, nom_pere, prenom_pere, nom_pere, nom_mere, source)
    site('delete', titre=titre, token=site.token())
    #site('edit', title=titre, text=text, token=site.token())

Définition de la source des données utilisées (qui est la même pour toutes les entrées).

In [10]:
source = 'https://mipade.mediancer.com/avl/avl_viewer.html?mag=14918'
titre = 'TestGenealogyBot3'

Création d'une page pour tous les chefs de famille dont la page n'est pas déjà existante. On utilise le fichier "1832_pc_relevant_columns.csv" qui contient le nom, le prénom et la date de naissance de tous les chefs de famille (y compris ceux qui n'ont pas d'enfants).

In [ ]:
data_bottin = pd.read_csv('../data/1832_pc_relevant_columns.csv')
nombre_chefs_famille = len(data_bottin.index)

for i in range (0,nombre_chefs_famille):
    prenom = data_bottin['chef_prenom'].str.title()[i]
    nom = data_bottin['chef_nom'].str.title()[i]
    titre = '[[%s %s]]'%(data_bottin['chef_prenom'].str.title()[i], data_bottin['chef_nom'].str.title()[i])

    new_page_father(titre, prenom, nom, date_naissance, source)

Création d'une page pour tous les enfants dont la page n'est pas déjà existante. On utilise le fichier "chef_enfant.csv" qui contient le prénom de tous les enfants associé à celui de leurs parents ainsi que leur date de naissance.

In [ ]:
data_bottin = pd.read_csv('../data/chef_enfant.csv')
nombre_enfants = len(data_bottin.index)

for i in range (0, nombre_enfants):
    
    prenom_enfant = data_bottin['enfants_dans_la_commune_prenom'].str.title()[i]
    date_naissance_enfant = data_bottin['annee_enfant'][i]
    nom_pere = data_bottin['chef_nom'].str.title()[i]
    prenom_pere = data_bottin['chef_prenom'].str.title()[i]
    nom_mere = data_bottin['epouse_nom'].str.title()[i]
    titre = '%s %s'%(data_bottin['enfants_dans_la_commune_prenom'].str.title()[i], data_bottin['chef_nom'].str.title()[i])
    
    new_page_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source)

#### 3.1.2 Création d'entrées correspondant à la naissance d'un enfant sur la page du chef de famille

Création de la fonction "entry_birth_children" permettant de rajouter à la page du chef de famille une entrée correpondant à la naissance de chacun de ses enfants. La syntaxe utilisée pour ces entrées est exactement la même que celle utilisée lors de la création de la page pour les enfants :

`[[Date]]/ -. [[Naissance]] de [[Prénom Nom]], enfant de [[Prénom Nom]] et de son épouse née Nom. [Source]`

In [ ]:
def entry_birth_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, source):
    text = '\n* [[%d]] / -. [[Naissance]] de [[%s %s]], enfant de [[%s %s]] et de son épouse née %s. [%s]\n'%(date_naissance_enfant, prenom_enfant, nom_pere, prenom_pere, nom_pere, nom_mere, source)

    #site('edit', title=titre, appendtext=text, token=site.token())

Créations des entrées correspondant à la naissance d'un enfant sur la page du père de famille.

In [ ]:
data_bottin = pd.read_csv('../data/chef_enfant.csv')
nombre_enfants = len(data_bottin.index)

for i in range (0, nombre_enfants):
    
    prenom_enfant = data_bottin['enfants_dans_la_commune_prenom'].str.title()[i]
    date_naissance_enfant = data_bottin['annee_enfant'][i]
    nom_pere = data_bottin['chef_nom'].str.title()[i]
    prenom_pere = data_bottin['chef_prenom'].str.title()[i]
    nom_mere = data_bottin['epouse_nom'].str.title()[i]
    titre = '%s %s'%(data_bottin['chef_prenom'].str.title()[i], data_bottin['chef_nom'].str.title()[i])
    

In [21]:
#Execution du programme
main()

SyntaxError: invalid syntax (<ipython-input-21-88b2bff89f9e>, line 1)

## Remarques utiles

Remarque :

- les dates sont des nombres dans "chef_enfant.csv" 
- les dates sont des strings dans "1832_pc_relevant_columns.csv"

In [ ]:
data_bottin = pd.read_csv('../data/chef_enfant.csv')
nombre_enfants = len(data_bottin.index)

date_naissance_enfant = data_bottin['annee_enfant'][0]
date_naissance_enfant


In [ ]:
data_bottin = pd.read_csv('../data/1832_pc_relevant_columns.csv')
nombre_chefs_famille = len(data_bottin.index)

date_naissance = data_bottin['chef_annee_naissance'][0]        
date_naissance